In [ ]:
import pandas as pd
import pandasql as ps
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
patients = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/patients.csv")
patients = patients.set_index("subject_id")
print(patients.head(200))

In [ ]:
admissions = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/admissions.csv")
admissions = admissions.set_index("subject_id")
print(admissions.head(200))

In [4]:
all_admissions = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/admissions.csv")
all_admissions = all_admissions.set_index("subject_id")

In [ ]:
all_admissions.head(200)

In [ ]:
diagnoses_icd = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/diagnoses_icd.csv")
print(diagnoses_icd.head(200))

In [ ]:
d_icd_diagnoses = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/d_icd_diagnoses.csv")
print(d_icd_diagnoses.head(200))

In [ ]:
d_icd_procedures = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/d_icd_procedures.csv")
print(d_icd_procedures.head(200))

In [ ]:
procedures_icd = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/procedures_icd.csv")
print(procedures_icd.head(200))

In [ ]:
drgcodes = pd.read_csv("/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/drgcodes.csv")
print(drgcodes.head(200))

In [ ]:
'''sql_query = """
SELECT COUNT(DISTINCT subject_id)
FROM admissions


"""
print(ps.sqldf(sql_query,locals()))'''

In [ ]:
'''sql_query = """
SELECT COUNT(DISTINCT subject_id)
FROM all_admissions


"""
print(ps.sqldf(sql_query,locals()))'''

# Aggregation of diagnoses


In [11]:
# Group diagnose descriptions (both of icd 9 and 10) by subject id need to join diagnoses_icd table with d_icd_diagnose to get the text description of which patient got which diagnose ...
sql_query = """
SELECT
    a.subject_id,
    GROUP_CONCAT(CASE WHEN di.icd_version = 9 THEN long_title END, ',') AS icd9Diagnoses,
    GROUP_CONCAT(CASE WHEN di.icd_version = 10 THEN long_title END, ',') AS icd10Diagnoses
FROM admissions a
JOIN diagnoses_icd di USING (subject_id)
JOIN d_icd_diagnoses USING (icd_code)
GROUP BY a.subject_id
"""
diagnoses_aggr = ps.sqldf(sql_query, locals())


# Aggregation for Procedures
same procedure as with icd diagnoses

In [12]:
# same as above...
sql_query = """
SELECT
    a.subject_id,
    GROUP_CONCAT(CASE WHEN pi.icd_version = 9 THEN long_title END, ',') AS icd9Procedures,
    GROUP_CONCAT(CASE WHEN pi.icd_version = 10 THEN long_title END, ',') AS icd10Procedures
FROM admissions a
LEFT JOIN procedures_icd pi USING (subject_id)
LEFT JOIN d_icd_procedures  USING (icd_code)
GROUP BY a.subject_id
"""

procedures_aggr = ps.sqldf(sql_query, locals())


In [13]:
# merging diagnoses and procedures
sql_query ="""
    SELECT *
    FROM procedures_aggr
    JOIN diagnoses_aggr USING (subject_id)
"""



procedAndDiagnoses_aggr = ps.sqldf(sql_query, locals())

# Adding Label (ground truth) for Data
1 = died
0 = survived

In [ ]:
sql_query = """
SELECT
    subject_id,
    icd9Procedures,
    icd10Procedures,
    icd9Diagnoses,
    icd10Diagnoses,
    drgDescription,
    CASE WHEN p.dod IS NOT NULL THEN 1 ELSE 0 END AS died
FROM procedAndDiagnosesAndDRG_aggr
JOIN patients p USING (subject_id)
"""
procedAndDiagnosesAndDRGAndDeaths_aggr = ps.sqldf(sql_query, locals())

In [ ]:
sql_query = """
SELECT
COUNT(subject_id)
FROM procedAndDiagnosesAndDRGAndDeaths_aggr"""

print(ps.sqldf(sql_query,locals()))

In [ ]:
procedAndDiagnosesAndDRGAndDeaths_aggr.head(200)

In [111]:
# save output
output_path = "/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/diag_proc_drg_deaths_aggr.csv"
procedAndDiagnosesAndDRGAndDeaths_aggr.to_csv(output_path, index=False)

In [112]:

import pandas as pd
import pandasql as ps
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')




In [114]:
# initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [115]:
# load data
path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/diag_proc_drg_deaths_aggr.csv'
allStringValues = pd.read_csv(path)

In [116]:
# check the class distribution
sql_query_died = "SELECT COUNT(*) AS died_count FROM allStringValues WHERE died = 1"
sql_query_survived = "SELECT COUNT(*) AS survived_count FROM allStringValues WHERE died = 0"
died_count = ps.sqldf(sql_query_died, locals()).iloc[0]['died_count']
survived_count = ps.sqldf(sql_query_survived, locals()).iloc[0]['survived_count']

In [ ]:
# num of deaths = 36872
# num of survived = 186419
print(f"Number of deaths: {died_count}")
print(f"Number of survived: {survived_count}")

In [118]:
# shuffle the dataset
survived_rows = allStringValues[allStringValues['died'] == 0].sample(frac=1, random_state=82)
# sample survived rows with num of died rows
survived_sample = survived_rows.sample(n=died_count, random_state=52)
# extract died rows
died_rows = allStringValues[allStringValues['died'] == 1]
# combine dataset
balanced_df = pd.concat([died_rows, survived_sample]).sample(frac=1, random_state=23)


In [ ]:
# Verify the balanced df
print(balanced_df['died'].value_counts())

In [ ]:
# added this code to reduce dataset in order to speed up fine tuning ....
hboolean = True  #set to false to keep origin dataset

if hboolean:
    print("Reducing dataset to 500 samples per category...")
    balanced_df = pd.concat([
        balanced_df[balanced_df['died'] == 0].sample(n=1000, random_state=62),
        balanced_df[balanced_df['died'] == 1].sample(n=1000, random_state=49)
    ])
    print(balanced_df['died'].value_counts())  # Verify ds
else:
    print("Keeping the dataset as is.")

# Text processing

In [121]:
# Rename died to encoded text for BERT
balanced_df['encoded_text'] = balanced_df['died']
balanced_df = balanced_df.drop(columns=['died'])


In [122]:
# combine  columns into one text column
balanced_df['text'] = balanced_df.drop(columns=['subject_id', 'encoded_text']).apply(
    lambda row: ', '.join([str(val) for val in row if pd.notna(val)]), axis=1
)

In [123]:
# keep only necessary columns
balanced_df = balanced_df[['subject_id', 'encoded_text', 'text']]

In [ ]:
# verify
balanced_df.head(100)

In [125]:
# preprocessing function lemmatization and stop words

In [126]:
def preprocess_text(text):
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
    return "".join(words)

In [127]:
balanced_df['text'] = balanced_df['text'].apply(preprocess_text)

In [128]:
# save otput
output_path = '/Users/cc/Desktop/Work/Uni/Berner FH/DataSets/mimicCSV/mimic-iv-3.1/hosp/shortform/preprocessed_for_bert.csv'

balanced_df.to_csv(output_path, index=False)